In [1]:
from collections import Counter
from sklearn.model_selection import KFold
import numpy as np
from scipy.sparse import csr_matrix
import re
import os
import shutil

from IPython.display import clear_output
from collections import defaultdict

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from functions import *

from keras.models import Sequential, Model
from keras.losses import mean_absolute_error
from keras.layers import Embedding, LSTM, Bidirectional, Dense, Input, Masking, Lambda, concatenate
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback, LearningRateScheduler, Callback

Using TensorFlow backend.


In [2]:
DICT_SIZE = 300000
ACTIVATION = 'relu'
HIDDEN_SIZE = 128

In [3]:
hash_model = make_hash_model(
    DICT_SIZE,
    5,
    ACTIVATION,
    HIDDEN_SIZE
)

In [4]:
input_from_lines = lambda lines: make_hash_input_from_lines(
    lines,
    DICT_SIZE,
)

In [6]:
history = train_epoch(hash_model, 1, 0, "positive_hash_data_10K.tsv", "negative_hash_data_10K.tsv", input_from_lines)

Epoch 1/1
 434/7656 [>.............................] - ETA: 55:32 - loss: 0.5402 - mean_positive_score: 0.6201 - mean_negative_score: 0.3716 - mean_positive_var: 0.4257 - mean_negative_var: 0.1838 - acc: 0.3513 - mean_squared_error: 1.0563

KeyboardInterrupt: 

In [7]:
with open("positive_hash_data_10K.tsv") as positive:
    positive_lines = [next(positive) for i in range(2)]

In [8]:
with open("negative_hash_data_10K.tsv") as negative:
    negative_lines = [next(negative) for i in range(2)]

In [9]:
positive_inputs = input_from_lines(positive_lines)

In [10]:
hash_model.predict(positive_inputs)

array([[0.26291257],
       [0.71036327]], dtype=float32)

In [11]:
negative_inputs = input_from_lines(negative_lines)

In [12]:
hash_model.predict(negative_inputs)

array([[0.16988361],
       [0.4891213 ]], dtype=float32)

In [ ]:
test_data_generator = make_generator(DataGenerator(
    "positive_hash_data_10K.tsv",
    "negative_hash_data_10K.tsv",
    hash_model,
    BATCH_SIZE,
    BATCH_SIZE * 1
))
z = next(test_data_generator)